In [ ]:
#@title <b>Time Out Preventer (Advanced) </b></strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
#@markdown **Run this code to prevent Google Colab from Timeout**
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True)
if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button");
      btn.click()
      }

    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect");
      btn.click()
      }
    }

  setInterval(ClickConnect,60000)
  '''))

In [ ]:
from IPython.display import clear_output
!apt-get upgrade
!pip install spotdl==v4.2.1
!pip install pandas
from google.colab import drive
import requests
import pandas as pd
import base64
import time
import os

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Calculating upgrade... Done
The following packages have been kept back:
  libcudnn8 libcudnn8-dev libnccl-dev libnccl2
The following packages will be upgraded:
  base-files bash binutils binutils-common binutils-x86-64-linux-gnu bsdutils coreutils
  cuda-compat-12-2 cuda-keyring cuda-toolkit-12-config-common cuda-toolkit-config-common dpkg
  dpkg-dev e2fsprogs libbinutils libblkid1 libc-bin libctf-nobfd0 libctf0 libdpkg-perl libext2fs2
  libgnutls30 libldap-2.5-0 libmount1 libpam-modules libpam-modules-bin libpam-runtime libpam0g
  libperl5.34 libprocps8 libsmartcols1 libss2 libudev1 linux-libc-dev login logsave mount openssl
  passwd perl perl-base perl-modules-5.34 procps tar util-linux
45 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Need to get 60.4 MB of archives.
After this operation, 168 kB of additional disk space will be used.
Get:1 https://developer.download.nvidia.

In [ ]:
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
def get_access_token(client_id, client_secret):
    credentials = f"{client_id}:{client_secret}"
    b64_credentials = base64.b64encode(credentials.encode()).decode()
    token_url = "https://accounts.spotify.com/api/token"
    response = requests.post(token_url, headers={
        "Authorization": f"Basic {b64_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    }, data={"grant_type": "client_credentials"})
    return response.json()['access_token']

In [ ]:
def get_playlist_tracks(playlist_id, access_token):
    tracks = []
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    while url:
        response = requests.get(url, headers={
            "Authorization": f"Bearer {access_token}"
        })
        data = response.json()
        tracks.extend(data['items'])
        url = data['next']  # Get the next page of results
        time.sleep(2)  # To avoid hitting the rate limit
    return [item['track'] for item in tracks if item['track'] is not None]

In [ ]:
# Function to get audio features for a list of track IDs
def get_audio_features(tracks, access_token):
    features_url = "https://api.spotify.com/v1/audio-features"
    audio_features_data = []
    i=0
    for track in tracks:
        track_id = track['id']
        track_name = track['name']
        response = requests.get(f"{features_url}/{track_id}", headers={
            "Authorization": f"Bearer {access_token}"
        })

        # Parse the response
        features = response.json()

        # Check if features is not None and id is not None
        if features and features.get('id') is not None:
            audio_data = {
                "id": features['id'],
                "name": track_name,
                "danceability": features['danceability'],
                "acousticness": features['acousticness'],
                "energy": features['energy'],
                "instrumentalness": features['instrumentalness'],
                "liveness": features['liveness'],
                "valence": features['valence'],
                "loudness": features['loudness'],
                "speechiness": features['speechiness'],
                "tempo": features['tempo'],
                "key": features['key'],
                "time_signature": features['time_signature']
            }
            audio_features_data.append(audio_data)
            print("retreived for:"+track_name)

        time.sleep(2)  # To avoid hitting the rate limit

    return audio_features_data

In [ ]:
client_id = ''  # Replace with your Spotify client ID
client_secret = ''  # Replace with your Spotify client secret

In [ ]:
spotify_playlist_url = input("Enter the Spotify playlist URL: ")
playlist_id = spotify_playlist_url.split('/')[-1].split('?')[0]
access_token = get_access_token(client_id, client_secret)

Enter the Spotify playlist URL: https://open.spotify.com/playlist/2IS2rrB2oyODqDkn79y13m?si=PCrS5wkRT6qgWdWlUfNl4w


In [ ]:
tracks = get_playlist_tracks(playlist_id, access_token)
audio_features = get_audio_features(tracks, access_token)
df = pd.DataFrame(audio_features)
output_dir = "/gdrive/MyDrive/"
os.makedirs(output_dir, exist_ok=True)
output_file_path = os.path.join(output_dir, "t5.csv")
df.to_csv(output_file_path, index=False)
print(f"Audio features saved to {output_file_path}")

retreived for:Kotha Raja
retreived for:Kalapakkaara
retreived for:Ee Ulakin
retreived for:King of Kotha - Title Track
retreived for:Doore
retreived for:People of Kotha
retreived for:King of Kotha (Teaser Theme)
retreived for:Kalapakkaara - Karaoke
retreived for:Beautifull Zindagi
retreived for:Idhera
retreived for:Nuvvu Nuvvu Kadhu
retreived for:O Kala
retreived for:Yevade Subramanyam
retreived for:Challa Gaali Thakuthunna
retreived for:Parvathi Thanayudavo
retreived for:Edo Jarige
retreived for:Deham Ichina Daivama
retreived for:Nedhi Nadhi Okae Kadha
retreived for:Naaloni Nuvvu Neeloni Nenu
retreived for:Settlement
retreived for:Nippai Ragile
retreived for:Reppalaninda
retreived for:Adire Hrudayam
retreived for:Pillaa Raa
retreived for:Rudhiram Marigi
retreived for:Manasunipatti Daaram Katti
retreived for:Dhinaku Dhina Da
retreived for:Le Le Lele
retreived for:Chilakamma Mukkuki
retreived for:Emantaro
retreived for:Killi Killi
retreived for:Chitti Nadumune
retreived for:Chiguraku Cha